In [1]:
import pathlib

In [2]:
import pandas as pd

In [3]:
root_dir_path = pathlib.Path("..")
data_dir_path = root_dir_path / "data"
raw_dir_path = data_dir_path / "raw"
sample_submission_path = raw_dir_path / "sample_submission.csv"
models_dir_path = root_dir_path / "models"
prediction_path = models_dir_path / "prediction.parquet"
submission_path = models_dir_path / "submission.csv.gz"

In [4]:
sample_submission = pd.read_csv(sample_submission_path)

In [5]:
prediction = pd.read_parquet(prediction_path)

In [6]:
# See https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/141452
prediction["demand"] *= 1.04

In [7]:
train_days = 1913
test_days = 28

In [8]:
d = prediction["d"].str[2:]
d = d.astype("int32")
d -= train_days + 1

is_evaluation = d >= test_days
d %= test_days

prediction["d"] = d
prediction.loc[is_evaluation, "id"] = prediction.loc[is_evaluation, "id"].str.replace(
    "_validation", "_evaluation"
)

In [9]:
submission = pd.pivot(prediction, index="id", columns="d", values="demand")

In [10]:
submission.reset_index(inplace=True)

In [11]:
submission.columns = ["id"] + [f"F{i + 1}" for i in range(test_days)]

In [12]:
submission = pd.merge(sample_submission["id"], submission, how="left")

In [13]:
submission.to_csv(submission_path, index=False)